In [1]:
%matplotlib inline

from PIL import Image
import glob
import pandas as pd
import matplotlib.pyplot as plt
good_pop = glob.glob('../screenshots/good/*.png')

bad_pop = glob.glob('../screenshots/bad/*.png')
print(len(good_pop), len(bad_pop))

460 239


In [2]:
from collections import Counter
from itertools import product
import colorsys
def normalize_pixel(pixel, num_color_steps=16):
    factor = (256//num_color_steps)
    return tuple(factor*(int((c)/factor)) for c in pixel)

def extract_features(image, num_channel_values=8):
    pixels_array = image.load()
    w, h = image.size
    
    color_counts = Counter(normalize_pixel(p) for p in image.getdata())
    num_pixels = w*h
    pixel_freqs = dict()
    common_pixel_f = (0,0)
    for k,v in color_counts.items():
        pixel_freqs.update({str(k):v/num_pixels})
        if v > common_pixel_f[1]:
            common_pixel = (k,v)
#     return pixel_freqs
#     print(color_counts.most_common(10))
    if len(color_counts) > 1:
        major_hue = colorsys.rgb_to_hsv(*(color_counts.most_common(2)[1][0]))[0]
    else:
        major_hue = colorsys.rgb_to_hsv(*(color_counts.most_common(2)[0][0]))[0]
    return {
        'num_colors': len(color_counts),
        'major_hue': major_hue,
        'empty_proportion': color_counts.most_common(1)[0][1]/(w*h),
        
    }

def extract_file(path):
    try:
        return extract_features(Image.open(path))
    except Exception as e:
        print("Error with path={}\n{}".format(path,e))
        return None

extract_features(Image.open(good_pop[0]))

{'empty_proportion': 0.9853221450617284,
 'major_hue': 0.8333333333333334,
 'num_colors': 257}

In [ ]:
from multiprocessing import Pool
import time
pool = Pool()

In [ ]:
ti = time.time()
bad_set = list(pool.map(extract_file, bad_pop))
tf = time.time()
print(tf-ti)
bad_frame = pd.DataFrame(bad_set)
bad_frame.describe()

In [ ]:
bad_frame.hist(column='empty_proportion')

In [ ]:
ti = time.time()
good_set = list(pool.map(extract_file, good_pop))
tf = time.time()
print(tf-ti)
good_frame = pd.DataFrame(good_set)
good_frame.describe()

In [ ]:
good_frame.hist(column='empty_proportion')